In [1]:
param_date = '20240715'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/9.type'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['1.ABARTH.xlsx']

In [3]:
file = list_file[0]

file

'1.ABARTH.xlsx'

In [4]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [5]:
input_ = pd.read_excel(f'./{param_date}/9.type/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Make No'] = input_['Make No'].astype(int)
input_['Model No'] = input_['Model No'].astype(int)
input_['Engine No'] = input_['Engine No'].astype(int)
input_['Year'] = input_['Year'].astype(int)
input_['Group No'] = input_['Group No'].astype(int)
input_['Type No'] = input_['Type No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Type'].tolist()

总数量：3


['Brake Hose\n', 'Alternator / Generator\n', 'Spark Plug']

In [6]:
a = 1

a

1

In [7]:
crawler_status = 'error'

crawler_status

'error'

In [8]:
import requests
import json

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'jsn': {'tab': 'catalog',
                   'make': input_.loc[a, 'Make'],
                   'year': int(input_.loc[a, 'Year']),
                   'model': input_.loc[a, 'Model'],
                   'carcode': input_.loc[a, 'Car Code'],
                   'engine': input_.loc[a, 'Engine'],
                   'groupname': input_.loc[a, 'Group'],
                   'parttype': input_.loc[a, 'Type Code'],
                   'nodetype': 'parttype'}}

data = {'func': 'navnode_fetch',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                 data=data,
                                 headers=get_header(),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1


<Response [200]>

In [9]:
json_ = resp.json()

json_

{'html_fill_sections': {'navchildren[]': '<div id="listings[__GIP__1__]" class="listings-container"><form method="post" id="frmlistings[__GIP__1__]" class="nobmp" action="/en/catalog/acura,2016,ilx,2.4l+l4+,3314496,+electrical%0A,alternator+/+generator,2412"><input type="hidden" name="_nck" value="8LVLUxTaILBT+tLxkg8+3Ag7/0bbOhvn7P3rkFbL80/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2anmEejEfH6DkMxNDgQRV4PP5CQNMBUg5tWYAVI8wKVT6Pmlj04vMyG"><div class="listing-container-border"><input type="hidden" autocomplete="off" id="parentnode[__GIP__1__]" value="" /><div><div style="padding-top: 1px; padding-left: .25em; padding-right: .25em;"><div class="remove-for-print" style="padding-top:5px;"><div class="ib vtop"><span style="padding-left: .5em;" class="show-if-js"><img onclick="cataloglite.NavNode_SimulateClick(null);" style="vertical-align: top;" class="mouseover" src="/catalog/images/close.gif" alt="Close" title="Close" /></span><span style=

In [10]:
'redirect_to_url' in json_

False

In [11]:
json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['year'] == input_.loc[a, 'Year'] and json_['jsn']['model'] == input_.loc[a, 'Model'] and json_['jsn']['carcode'] == input_.loc[a, 'Car Code'] and json_['jsn']['engine'] == input_.loc[a, 'Engine'] and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['parttype'] == input_.loc[a, 'Type Code'] and json_['jsn']['nodetype'] == 'parttype' and 'collected_javascript' in json_

True

In [12]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'].strip(), 'lxml')
html = etree.HTML(str(soup))

with open('./part_1.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="listings-container" id="listings[__GIP__1__]">
   <form action="/en/catalog/acura,2016,ilx,2.4l+l4+,3314496,+electrical%0A,alternator+/+generator,2412" class="nobmp" id="frmlistings[__GIP__1__]" method="post">
    <input name="_nck" type="hidden" value="8LVLUxTaILBT+tLxkg8+3Ag7/0bbOhvn7P3rkFbL80/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2anmEejEfH6DkMxNDgQRV4PP5CQNMBUg5tWYAVI8wKVT6Pmlj04vMyG"/>
    <div class="listing-container-border">
     <input autocomplete="off" id="parentnode[__GIP__1__]" type="hidden" value=""/>
     <div>
      <div style="padding-top: 1px; padding-left: .25em; padding-right: .25em;">
       <div class="remove-for-print" style="padding-top:5px;">
        <div class="ib vtop">
         <span class="show-if-js" style="padding-left: .5em;">
          <img alt="Close" class="mouseover" onclick="cataloglite.NavNode_SimulateClick(null);" src="/catalog/images/close.gif" style="vertical-al

In [13]:
list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

list_tbody

[<Element tbody at 0x10f292b00>,
 <Element tbody at 0x127bf2400>,
 <Element tbody at 0x10f2b4ac0>,
 <Element tbody at 0x127e853c0>,
 <Element tbody at 0x127e85c00>,
 <Element tbody at 0x127e85f40>,
 <Element tbody at 0x127e85a80>]

In [14]:
len(list_tbody) == 0

False

In [15]:
list_list_note = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()') for tbody in list_tbody]
list_note = ['\n'.join([note.strip() for note in list_note]) for list_note in list_list_note]

list_note

['',
 'Reman',
 'Remanufactured',
 'Remanufactured',
 'Gold; Remanufactured',
 'Remanufactured',
 'Reman']

In [16]:
list_list_info = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href') for tbody in list_tbody]
list_info = [list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else '' for list_info in list_list_info]

list_info

['https://www.rockauto.com/en/moreinfo.php?pk=12807505&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=11503749&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=7471464&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=13459361&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=12162449&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=7314976&cc=3314496&pt=2412',
 'https://www.rockauto.com/en/moreinfo.php?pk=11195941&cc=3314496&pt=2412']

In [17]:
list_dict_note = []
list_span = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]')[0] for tbody in list_tbody]

for span in list_span:
    dict_note = {}
    list_note_2 = span.xpath('./span[@class="listing-footnote-text"]/text()')
    list_market = span.xpath('./span[@class="listing-footnote-text"]/a[@class="appflag"]/@data-market')
    if list_market:
        dict_note['separate'] = {}
        for i, note, market in zip(range(len(list_note_2)), list_note_2, list_market):
            dict_note['separate'][str(i)] = {'market': market.strip(), 'note': note.strip()}
    else:
        dict_note = {'combined': {'note': ''.join(list_note_2).strip()}}

    list_dict_note.append(dict_note)

list_dict_note

[{'combined': {'note': ''}},
 {'combined': {'note': '135 Amps'}},
 {'combined': {'note': '135 Amps'}},
 {'combined': {'note': ''}},
 {'combined': {'note': 'Pulley 7SCP; Amps: 125'}},
 {'combined': {'note': '125 Amps'}},
 {'combined': {'note': '130 Amp'}}]

In [18]:
list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

list_json

[{'groupindex': '__GIP__2__',
  'carcode': '3314496',
  'parttype': '2412',
  'partkey': '12807505',
  'opts': {'0-0-0-1': {'warehouse': '11702',
    'whpartnum': 'TY 2-21154',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}},
 {'groupindex': '__GIP__3__',
  'carcode': '3314496',
  'parttype': '2412',
  'partkey': '11503749',
  'opts': {'0-0-0-1': {'warehouse': '44070',
    'whpartnum': 'BBB 42084',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}},
 {'groupindex': '__GIP__4__',
  'carcode': '3314496',
  'parttype': '2412',
  'partkey': '7471464',
  'opts': {'0-0-0-1': {'warehouse': '13209',
    'whpartnum': 'REM 11227',
    'optionlist': '0',
    'paramcode': '0',
    'notekey': '0',
    'multiple': '1'}}},
 {'groupindex': '__GIP__5__',
  'carcode': '3314496',
  'parttype': '2412',
  'partkey': '13459361',
  'opts': {'0-0-0-1': {'warehouse': '80011',
    'whpartnum': 'FVP 10268',
    'optionlist': '0',
    'p

In [19]:
list_key = [json_['partkey'] for json_ in list_json]

list_key

['12807505',
 '11503749',
 '7471464',
 '13459361',
 '12162449',
 '7314976',
 '11195941']

In [20]:
list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

list_json

[{'partnumber': '221154',
  'catalogname': 'TYC',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 0,
  'sortgrouptext': 'New',
  'paramdesc': ''},
 {'partnumber': '42084',
  'catalogname': 'BBB INDUSTRIES',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 1,
  'sortgrouptext': 'Remanufactured',
  'paramdesc': ''},
 {'partnumber': '11227',
  'catalogname': 'REMY',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 1,
  'sortgrouptext': 'Remanufactured',
  'paramdesc': ''},
 {'partnumber': '10268',
  'catalogname': 'FVP',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 1,
  'sortgrouptext': 'Remanufactured',
  'paramdesc': ''},
 {'partnumber': '3343085',
  'catalogname': 'ACDELCO',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 1,
  'sortgrouptext': 'Remanufactured',
  'paramdesc': ''},
 {'partnumber': '10268',
  'catalogname': 'PURE ENERGY',
  'belongstolisting': '__GIP__1__',
  'sortgroup': 1,
  'sortgrouptext': 'Remanufactured',
  'paramdesc': ''},
 {'partnumber': '2101212

In [21]:
list_part_number = [json_['partnumber'] for json_ in list_json]

list_part_number

['221154', '42084', '11227', '10268', '3343085', '10268', '2101212']

In [22]:
list_manufacturer = [json_['catalogname'] for json_ in list_json]

list_manufacturer

['TYC', 'BBB INDUSTRIES', 'REMY', 'FVP', 'ACDELCO', 'PURE ENERGY', 'DENSO']

In [23]:
list_partition = [json_['sortgrouptext'] for json_ in list_json]

list_partition

['New',
 'Remanufactured',
 'Remanufactured',
 'Remanufactured',
 'Remanufactured',
 'Remanufactured',
 'Remanufactured']

In [24]:
list_parameter = [json_['paramdesc'] for json_ in list_json]

list_parameter

['', '', '', '', '', '', '']

In [25]:
list_list_oem = [tbody.xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()') for tbody in list_tbody]
list_oem = [','.join(list_oem) for list_oem in list_list_oem]
list_oem = [';'.join([oem_split.strip() for oem_split in oem.split(',')]) for oem in list_oem]

list_oem

['063115X6505RM;311005X6J01;311005X6J01RM',
 '311005X6J01;CSP47;90295874',
 '311005X6J01;CSP47',
 '311005X6J01;311005X6J01RM;CSP47',
 '',
 '311005X6J01;311005X6J01RM;CSP47;MPA10268',
 '1042113470;2100809;311005X6J01;311005X6J01RM;CSP47']

In [26]:
list_list_src = [json.loads(tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') else [] for tbody in list_tbody]

list_list_src

[[{'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/41/large_2-21154_1wm__ra_m.jpg',
    'Popup': '/info/41/large_2-21154_1wm.jpg',
    'Full': '/info/41/large_2-21154_1wm.jpg'}},
  {'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/41/large_2-21154_2wm__ra_m.jpg',
    'Popup': '/info/41/large_2-21154_2wm.jpg',
    'Full': '/info/41/large_2-21154_2wm.jpg'}},
  {'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/41/large_2-21154_3wm__ra_m.jpg',
    'Popup': '/info/41/large_2-21154_3wm.jpg',
    'Full': '/info/41/large_2-21154_3wm.jpg'}},
  {'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'ImageData': {'Thumb': '/info/41/large_2-21154_4wm__ra_m.jpg',
    'Popup': '/info/41/large_2-21154_4wm.jpg',
    'Full': '/info/41/large_2-21154_4wm.jpg'}},
  {'TitlePhrase': None,
   'ParamCodes': [None],
   'Type': 'Single',
   'Im

In [27]:
list_dict_src = []
for list_src in list_list_src:
    dict_src = {}
    for i, src in zip(range(len(list_src)), list_src):
        dict_src[str(i)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()

    list_dict_src.append(dict_src)

list_dict_src

[{'0': 'https://www.rockauto.com/info/41/large_2-21154_1wm.jpg',
  '1': 'https://www.rockauto.com/info/41/large_2-21154_2wm.jpg',
  '2': 'https://www.rockauto.com/info/41/large_2-21154_3wm.jpg',
  '3': 'https://www.rockauto.com/info/41/large_2-21154_4wm.jpg',
  '4': 'https://www.rockauto.com/info/41/large_2-21154_5wm.jpg'},
 {'0': 'https://www.rockauto.com/info/71/42084_ANG.jpg',
  '1': 'https://www.rockauto.com/info/71/42084_FRO.jpg',
  '2': 'https://www.rockauto.com/info/71/42084_BAC.jpg',
  '3': 'https://www.rockauto.com/info/71/42084_SID.jpg'},
 {'0': 'https://www.rockauto.com/info/177/11227_ANG.jpg',
  '1': 'https://www.rockauto.com/info/177/11227_FRO.jpg',
  '2': 'https://www.rockauto.com/info/177/11227_BAC.jpg',
  '3': 'https://www.rockauto.com/info/177/11227_SID.jpg',
  '4': 'https://www.rockauto.com/info/177/11110_CON.jpg'},
 {'0': 'https://www.rockauto.com/info/466/10268-ANG.jpg',
  '1': 'https://www.rockauto.com/info/466/10268-BAC.jpg',
  '2': 'https://www.rockauto.com/info/

In [28]:
df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                        'Make': input_.loc[a, 'Make'],
                        'Model No': input_.loc[a, 'Model No'],
                        'Model': input_.loc[a, 'Model'],
                        'Engine No': input_.loc[a, 'Engine No'],
                        'Engine': input_.loc[a, 'Engine'],
                        'Car Code': input_.loc[a, 'Car Code'],
                        'Year': input_.loc[a, 'Year'],
                        'Group No': input_.loc[a, 'Group No'],
                        'Group': input_.loc[a, 'Group'],
                        'Type No': input_.loc[a, 'Type No'],
                        'Type': input_.loc[a, 'Type'],
                        'Type Code': input_.loc[a, 'Type Code'],
                        'Tab 1 Url': input_.loc[a, 'Tab 1 Url'],
                        'Part No': [i+1 for i in range(len(list_part_number))],
                        'Partition': list_partition,
                        'Manufacturer': list_manufacturer,
                        'Part Number': list_part_number,
                        'Key': list_key,
                        'JOIN_MPNTCK': [f'''{manufacturer};{part_number};{input_.loc[a, 'Type Code']};{key}''' for manufacturer, part_number, key in zip(list_manufacturer, list_part_number, list_key)],
                        'Tab 1 Note 1': list_note,
                        'Tab 1 Note 2': [json.dumps(dict_note) for dict_note in list_dict_note],
                        'Parameter': list_parameter,
                        'OEM': list_oem,
                        'Info': list_info,
                        'Json_Src': [json.dumps(dict_src) for dict_src in list_dict_src]})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Make No,Make,Model No,Model,Engine No,Engine,Car Code,Year,Group No,Group,...,Manufacturer,Part Number,Key,JOIN_MPNTCK,Tab 1 Note 1,Tab 1 Note 2,Parameter,OEM,Info,Json_Src
0,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,TYC,221154,12807505,TYC;221154;2412;12807505,,"{""combined"": {""note"": """"}}",,063115X6505RM;311005X6J01;311005X6J01RM,https://www.rockauto.com/en/moreinfo.php?pk=12...,"{""0"": ""https://www.rockauto.com/info/41/large_..."
1,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,BBB INDUSTRIES,42084,11503749,BBB INDUSTRIES;42084;2412;11503749,Reman,"{""combined"": {""note"": ""135 Amps""}}",,311005X6J01;CSP47;90295874,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""https://www.rockauto.com/info/71/42084_..."
2,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,REMY,11227,7471464,REMY;11227;2412;7471464,Remanufactured,"{""combined"": {""note"": ""135 Amps""}}",,311005X6J01;CSP47,https://www.rockauto.com/en/moreinfo.php?pk=74...,"{""0"": ""https://www.rockauto.com/info/177/11227..."
3,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,FVP,10268,13459361,FVP;10268;2412;13459361,Remanufactured,"{""combined"": {""note"": """"}}",,311005X6J01;311005X6J01RM;CSP47,https://www.rockauto.com/en/moreinfo.php?pk=13...,"{""0"": ""https://www.rockauto.com/info/466/10268..."
4,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,ACDELCO,3343085,12162449,ACDELCO;3343085;2412;12162449,Gold; Remanufactured,"{""combined"": {""note"": ""Pulley 7SCP; Amps: 125""}}",,,https://www.rockauto.com/en/moreinfo.php?pk=12...,"{""0"": ""https://www.rockauto.com/info/333/19427..."
5,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,PURE ENERGY,10268,7314976,PURE ENERGY;10268;2412;7314976,Remanufactured,"{""combined"": {""note"": ""125 Amps""}}",,311005X6J01;311005X6J01RM;CSP47;MPA10268,https://www.rockauto.com/en/moreinfo.php?pk=73...,"{""0"": ""https://www.rockauto.com/info/466/10268..."
6,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,DENSO,2101212,11195941,DENSO;2101212;2412;11195941,Reman,"{""combined"": {""note"": ""130 Amp""}}",,1042113470;2100809;311005X6J01;311005X6J01RM;C...,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""https://www.rockauto.com/info/14/210-12..."


In [29]:
crawler_status = 'ok'

crawler_status

'ok'

In [30]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                             'Make': input_.loc[a, 'Make'],
                             'Model No': input_.loc[a, 'Model No'],
                             'Model': input_.loc[a, 'Model'],
                             'Engine No': input_.loc[a, 'Engine No'],
                             'Engine': input_.loc[a, 'Engine'],
                             'Car Code': input_.loc[a, 'Car Code'],
                             'Year': input_.loc[a, 'Year'],
                             'Group No': input_.loc[a, 'Group No'],
                             'Group': input_.loc[a, 'Group'],
                             'Type No': input_.loc[a, 'Type No'],
                             'Type': input_.loc[a, 'Type'],
                             'Type Code': input_.loc[a, 'Type Code'],
                             'Tab 1 Url': input_.loc[a, 'Tab 1 Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [31]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year', 'Group No', 'Type No', 'Part No'],
                                                ascending=[True, True, True, False, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part_1.xlsx', index=False)

output_correct

,Make No,Make,Model No,Model,Engine No,Engine,Car Code,Year,Group No,Group,...,Manufacturer,Part Number,Key,JOIN_MPNTCK,Tab 1 Note 1,Tab 1 Note 2,Parameter,OEM,Info,Json_Src
0,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,TYC,221154,12807505,TYC;221154;2412;12807505,,"{""combined"": {""note"": """"}}",,063115X6505RM;311005X6J01;311005X6J01RM,https://www.rockauto.com/en/moreinfo.php?pk=12...,"{""0"": ""https://www.rockauto.com/info/41/large_..."
1,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,BBB INDUSTRIES,42084,11503749,BBB INDUSTRIES;42084;2412;11503749,Reman,"{""combined"": {""note"": ""135 Amps""}}",,311005X6J01;CSP47;90295874,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""https://www.rockauto.com/info/71/42084_..."
2,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,REMY,11227,7471464,REMY;11227;2412;7471464,Remanufactured,"{""combined"": {""note"": ""135 Amps""}}",,311005X6J01;CSP47,https://www.rockauto.com/en/moreinfo.php?pk=74...,"{""0"": ""https://www.rockauto.com/info/177/11227..."
3,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,FVP,10268,13459361,FVP;10268;2412;13459361,Remanufactured,"{""combined"": {""note"": """"}}",,311005X6J01;311005X6J01RM;CSP47,https://www.rockauto.com/en/moreinfo.php?pk=13...,"{""0"": ""https://www.rockauto.com/info/466/10268..."
4,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,ACDELCO,3343085,12162449,ACDELCO;3343085;2412;12162449,Gold; Remanufactured,"{""combined"": {""note"": ""Pulley 7SCP; Amps: 125""}}",,,https://www.rockauto.com/en/moreinfo.php?pk=12...,"{""0"": ""https://www.rockauto.com/info/333/19427..."
5,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,PURE ENERGY,10268,7314976,PURE ENERGY;10268;2412;7314976,Remanufactured,"{""combined"": {""note"": ""125 Amps""}}",,311005X6J01;311005X6J01RM;CSP47;MPA10268,https://www.rockauto.com/en/moreinfo.php?pk=73...,"{""0"": ""https://www.rockauto.com/info/466/10268..."
6,1,ACURA,1,ILX,1,2.4L L4,3314496,2016,1,Electrical\n,...,DENSO,2101212,11195941,DENSO;2101212;2412;11195941,Reman,"{""combined"": {""note"": ""130 Amp""}}",,1042113470;2100809;311005X6J01;311005X6J01RM;C...,https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""https://www.rockauto.com/info/14/210-12..."


In [32]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year', 'Group No', 'Type No'],
                                            ascending=[True, True, True, False, True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_1_error.xlsx', index=False)

output_error

""
